In Python, you need to import a package before using it. It is a good practice to import the packages at the beginning of the code. There are packages for Python that are very commonly used that they are assigned an "alias" to be used throughout the code without having to write its complete name every time one wants to use a function from it. For instance NumPy and Pandas are two very commonly used libraries. They are commonly imported as:

```python
import numpy as np
import pandas as pd
```

For the rest of this part of the project, we will make use of three different libraries: `numpy`, `netCDF4` and `pandas`.

Sometimes you will see something like:

```python
from numpy import array
```

This means that we are only importing the module/class/function "array" from numpy. Any other class/function that are part of numpy will not be accesible/able to be used throughout the notebook. However, sometimes you only want to use one specific function from a package, and there is no need to load it fully. If you don't use an alias, to call the function you need to use the complete name, in this case `array`.

In [1]:
# Import the packages that we will use in this activity

import numpy as np
from netCDF4 import Dataset
import pandas as pd


In [2]:
"""
Write here the point coordinates corresponding to the
location that you want to analyze. The coordinates below
correspond to the location of Naches.
"""
lat0 = 46.7341
lon0 = -120.7146

In [3]:
"""
Climate model output is organized as "layers" and vectors of data. 
One layer, for instance, contains information about "precipitation" which is a tri-dimensional 
vector because it depends on the grid cell location and the time. 
In Python, you can save these vectors as lists, or numpy arrays.

The following function takes as arguments the lat, lon coordinates of 
the points that we specified above, and returns the list indeces of the 
grid cell center that is closest to the point that you specified.

"""


def naive_fast(latvar,lonvar,lat0,lon0):
    # Read latitude and longitude from file into numpy arrays
    latvals = latvar[:]
    lonvals = lonvar[:]
    ny,nx = latvals.shape
    dist_sq = (latvals-lat0)**2 + (lonvals-lon0)**2
    minindex_flattened = dist_sq.argmin()  # 1D index of min element
    iy_min,ix_min = np.unravel_index(minindex_flattened, latvals.shape)
    return iy_min,ix_min

In [4]:
"""
Here specify the path to the directory where you stored the
data of the future run that you downloaded from the GitHub page.

"""

filename = ('/Users/tanialopez/documents/cmu/spring19/12657/'
'Project2Part2/wrf-pgw-yakima/yakima_all/'
'yakima_PGW_PREC_ACC_NC_200010-201309.nc')

In [5]:
"""
Here specify the path to the directory where you stored the
data of the control run that you downloaded from the GitHub page.

"""


filenamec = ("/Users/tanialopez/documents/cmu/spring19/12657/"
             "Project2Part2/wrf-ctrl-yakima/"
             "yakima_CRTL_PREC_ACC_NC_200010-201309.nc")


In [6]:
"""

This line calls the class Dataset that we loaded into the notebook 
from the netCDF4 package at the beginning of the session. 
This class allows us to read the data stored in the .nc file.

Filename is the variable that contains the path where our .nc file is stored,
the "r" input means that we are reading the file and storing the content in "ncfile"

"""

ncfile = Dataset(filename, 'r')

In [8]:
"""
The following line outputs a summary of the variables for which data is
contained in the .nc file. 

Each variable is stored as a dictionary key 

"""
ncfile.variables

OrderedDict([('PREC_ACC_NC', <class 'netCDF4._netCDF4.Variable'>
              float32 PREC_ACC_NC(Time, south_north, west_east)
                  FieldType: 104
                  MemoryOrder: XY 
                  description: ACCUMULATED GRID SCALE  PRECIPITATION OVER prec_acc_dt PERIODS OF TIME
                  units: mm
                  stagger: 
                  coordinates: XLONG XLAT
                  long_name: PREC_ACC_NC
              unlimited dimensions: Time
              current shape = (113788, 63, 72)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('Time', <class 'netCDF4._netCDF4.Variable'>
              float64 Time(Time)
                  units: hours since 1901-01-01 00:00:00
                  calendar: standard
                  long_name: Time
                  description: Time
              unlimited dimensions: Time
              current shape = (113788,)
              filling on, default _FillValue of 9.96920996838

Take some time to read the output in the previous cell. 
The variables in a ncfile are stored as keys in the netCDF file.
For this particular file, there are five different variables.

`'PREC_ACC_NC'` which according to the description is the accumulated grid scale precipitation over a time delta. The timestep in the model is 60 minutes or 1 hour.

`'Time'` which according to the description and units, it is the number of ours since 1901-01-01 00:00:00, we can use this variable to create timestamps (MM-DD-YY HH:MM) for the precipitation happening at each hour of the simulated period using 1901-01-01 00:00:00 as the origin

`'XLAT'` which according to the description is the latitude of the gridcell centers of the simulation mesh. Note that the shape of this variable is a matrix (two dimensions). The reason is that the grid used in this model is not rectangular.

`'XLON'` same as `'XLAT'` but longitude






In [9]:
"""
Use this function to create timestamps for each precipitation record that will 
be useful later when we output the data as a timeseries.

"""

def get_timestamps(ncfile):
    """
    This function outputs a timestamp series using the Time variable from the ncfile, and its origin
    ---
    input: .nc file
    output: [array] timestamps starting from the .nc file Time variable origin
    """
    timev = ncfile.variables['Time'][:]
    return pd.to_datetime(timev, unit="h", origin=pd.Timestamp('1901-01-01 00:00:00'))

In [10]:
"""
Use the function defined earlier to obtain the timestamps

""" 

timenc = get_timestamps(ncfile)

In [11]:
"""
The following function takes as input the variable that contains 
the data from the .nc file (for our case, it's "ncfile".) and
the coordinates of your desired location for analysis. 

It outputs the coordinates of the grid cell center that is closest 
to the desired point.
"""

def get_coordinates(data, lat0, lon0):
    """
    This function outputs the coordinates of the closest grid cell center from the climate model simulation
    stored as a .nc file, to a point given its coordinates
    ---
    input:
    .nc file
    lat0 - float, latitude of the point of interest
    lon0 - float, longitude of the point of interest
    output: 
    tuple, lat, lon coordinates of the closest grid cell center to the point of interest
    """
    print('Finding coordinates of grid cell center closest to: {},{}'.format(lat0, lon0))
    lat_var = data.variables['XLAT']
    y = lat_var[:]
    y_nm = np.ma.getdata(y)
    lon_var = data.variables['XLONG']
    x = lon_var[:]
    x_nm = np.ma.getdata(x)
    
    iy_, ix_ = naive_fast(lat_var,lon_var,lat0,lon0)  
    
    print('Closest gridcell center coordinates: {},{}'.format(
        y_nm[iy_,ix_], x_nm[iy_,ix_]))
    
    return iy_, ix_

In [12]:
"""
Use function defined above to find the indeces of the gridcell 
center whose coordinates are closest to the point of interest

The indeces are two, because as we discussed earlier, the
grid used in this model is not rectangular. But instead curved, 
so each grid is not a perfect square.

"""

iy, ix = get_coordinates(ncfile, lat0, lon0)

Finding coordinates of grid cell center closest to: 46.7341,-120.7146
Closest gridcell center coordinates: 46.73405456542969,-120.714599609375


In [13]:
"""
This function extracts precipitation records for the gridcell 
center that we found earlier is closes to our point of interest.
"""

def extract_precipitation(ncfile, iy, ix):
    
    ## First read description of the ncfile to know what are the
    ## units of precipitation in the model run
    
    for k in ncfile.variables.keys():
        print('Variable: {}, Units: {}'.format(k, getattr(ncfile.variables[k], 'units')))
    
    ## Precipitation is in mm, no need to apply any conversion factor
    
    #now, extract precipitation for the point found earlier
    p = np.ma.getdata(ncfile.variables['PREC_ACC_NC'][:, iy, ix])
    
    ncfile.close()
    
    return p
    

In [14]:
"""
Use the function above to extract precipitation and store it
on variable "p"
"""

p = extract_precipitation(ncfile, iy, ix)

Variable: PREC_ACC_NC, Units: mm
Variable: Time, Units: hours since 1901-01-01 00:00:00
Variable: Times, Units: 1
Variable: XLAT, Units: degree_north
Variable: XLONG, Units: degree_east
Variable: XTIME, Units: minute


In [20]:
"""
Create timeseries pandas dataframe to store as csv and perform 
some basic statistics on the data using excel

We make use of the class DataFrame from the package pandas.
This package is extremely useful to do very efficient operations on large data.
It organizes the data into "tables". You can google it for more information 
about how dataframes (tables) are created using pandas and the different 
types of data in Python (lists, dictionaries, tuples, etc.)

"""

df_pgw = pd.DataFrame({'prcp': p, 'date': timenc})

In [21]:
"""
Using the following .tail method in the dataframe variable, 
you can take a look at how the last 5 rows of the dataframe look like!
Try doing .head() as well! """

df_pgw.tail()

,date,prcp
113783,2013-09-30 19:00:00,0.000091
113784,2013-09-30 20:00:00,0.001890
113785,2013-09-30 21:00:00,0.100738
113786,2013-09-30 22:00:00,0.032355
113787,2013-09-30 23:00:00,0.000000


In [22]:
"""
The folowing function takes as input the dataframe above 
and the frequency you would like to agregate the data to. 

For instance, we are now having hourly data, but if you would
like monthly data then we would need to aggregate precipitation (sum)
over each month. 

The variable col is the name of the column which has the timestamp"""


def aggregate_by_frequency(df, fr, col):
    """Create a new df with col aggregated by desired frequency
    Note: df needs to have a column named "date" and be a datetime object
    in order to work. Available freq depending on frequency of the original data
    possible values of freq: "3h, 6h, 12h, 24h..." and so on. It is not limited
    to hours but can also aggregate by week or by month, by year too."""

    return df.groupby(pd.Grouper(key='date', freq=fr))[[col]].sum()

In [24]:
"""
We can aggregate by year or by month.

To aggregate by year, you need to pass fr = 'A'
and to aggregate by month, you need to pass fr = 'M'

The different keys are stated here:

https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
"""

annual_prcp_pgw = aggregate_by_frequency(df_pgw, 'A', 'prcp')['2001':'2012']

In [25]:
annual_prcp_pgw

,prcp
date,
2001-12-31,293.229126
2002-12-31,347.835175
2003-12-31,344.923218
2004-12-31,293.310242
2005-12-31,447.366516
2006-12-31,442.266083
2007-12-31,285.201935
2008-12-31,334.667358
2009-12-31,344.671173


In [26]:
mpgw = aggregate_by_frequency(df_pgw, 'M', 'prcp')

In [28]:
mpgw['month'] = mpgw.index.month

In [100]:
mpgw.groupby('month').mean()

,prcp
month,
1,50.018436
2,33.337475
3,36.113968
4,27.368488
5,32.772182
6,21.042255
7,6.378665
8,5.734042
9,15.047806


In [29]:
mpgw

,prcp,month
date,,
2000-10-31,18.576715,10
2000-11-30,17.840023,11
2000-12-31,45.356262,12
2001-01-31,30.501490,1
2001-02-28,24.457994,2
2001-03-31,32.766254,3
2001-04-30,19.019220,4
2001-05-31,6.335215,5
2001-06-30,6.167444,6


In [32]:
"""
You can save the dataframes above as .csv file to analyze the data in Excel
using the following line.

"""


#insert here the name and path where you want to save your .csv file
filename_save = 'monthly_pgw.csv'
save_path = '~/Desktop'

savef = save_path + '/' + filename_save

mpgw.to_csv(savef)


### Repeat the same operations for the control run.
